In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128'

In [4]:
pretrained = ''
resume = './log/pggan/128/b1/checkpoint.pth.tar'

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 412
test_batch = 400
lr = 0.1
schedule = [20, 125, 225, 275]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b1' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

==> Resuming from checkpoint..


# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [252 | 300] LR: 0.000769
1/312 Data:2.560 | Batch:8.246 | Total:0:00:08 | ETA:0:42:45 | Loss:0.03397712856531143 | top1:97.57281494140625
11/312 Data:0.001 | Batch:0.774 | Total:0:00:16 | ETA:0:08:01 | Loss:0.03404067067260092 | top1:98.3671646118164
21/312 Data:0.009 | Batch:0.757 | Total:0:00:24 | ETA:0:03:41 | Loss:0.038700006902217865 | top1:98.42810821533203
31/312 Data:0.001 | Batch:0.721 | Total:0:00:31 | ETA:0:03:33 | Loss:0.034939258990268555 | top1:98.62981414794922
41/312 Data:0.005 | Batch:0.729 | Total:0:00:39 | ETA:0:03:30 | Loss:0.03407007623918173 | top1:98.6857681274414
51/312 Data:0.001 | Batch:0.754 | Total:0:00:47 | ETA:0:03:15 | Loss:0.03268885188827328 | top1:98.76261138916016
61/312 Data:0.004 | Batch:0.773 | Total:0:00:54 | ETA:0:03:09 | Loss:0.032825348959838754 | top1:98.77049255371094
71/312 Data:0.022 | Batch:0.774 | Total:0:01:02 | ETA:0:03:06 | Loss:0.03294406344615657 | top1:98.78640747070312
81/312 Data:0.029 | Batch:0.860 | Total:0:01:10 | ETA:0

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


81/81 Data:0.000 | Batch:0.414 | Total:0:00:21 | ETA:0:00:00 | Loss:0.015347983019362636 | top1:99.47975158691406

Epoch: [253 | 300] LR: 0.000170
1/312 Data:2.099 | Batch:3.542 | Total:0:00:03 | ETA:0:18:22 | Loss:0.040118537843227386 | top1:99.02912139892578
11/312 Data:0.003 | Batch:0.775 | Total:0:00:11 | ETA:0:05:14 | Loss:0.03769330036911098 | top1:98.83053588867188
21/312 Data:0.001 | Batch:0.792 | Total:0:00:18 | ETA:0:03:43 | Loss:0.03627399989359435 | top1:98.71705627441406
31/312 Data:0.001 | Batch:0.721 | Total:0:00:26 | ETA:0:03:35 | Loss:0.03579674117387303 | top1:98.63764953613281
41/312 Data:0.001 | Batch:0.748 | Total:0:00:33 | ETA:0:03:22 | Loss:0.03424548771141506 | top1:98.7094497680664
51/312 Data:0.001 | Batch:0.721 | Total:0:00:41 | ETA:0:03:22 | Loss:0.0353703320610757 | top1:98.65791320800781
61/312 Data:0.001 | Batch:0.781 | Total:0:00:49 | ETA:0:03:10 | Loss:0.033497440002736495 | top1:98.74662017822266
71/312 Data:0.011 | Batch:0.843 | Total:0:00:56 | ETA:0:

41/312 Data:0.004 | Batch:0.921 | Total:0:00:34 | ETA:0:03:29 | Loss:0.028834126480832334 | top1:98.9166488647461
51/312 Data:0.001 | Batch:0.725 | Total:0:00:41 | ETA:0:03:26 | Loss:0.029446037924465013 | top1:98.91014862060547
61/312 Data:0.001 | Batch:0.735 | Total:0:00:49 | ETA:0:03:07 | Loss:0.030110880396649487 | top1:98.87792205810547
71/312 Data:0.001 | Batch:0.798 | Total:0:00:56 | ETA:0:03:03 | Loss:0.03028923658732797 | top1:98.87187194824219
81/312 Data:0.001 | Batch:0.748 | Total:0:01:04 | ETA:0:03:00 | Loss:0.030751159506631487 | top1:98.84333801269531
91/312 Data:0.009 | Batch:0.738 | Total:0:01:11 | ETA:0:02:47 | Loss:0.030825997160358744 | top1:98.83441925048828
101/312 Data:0.001 | Batch:0.731 | Total:0:01:19 | ETA:0:02:37 | Loss:0.031115159595219214 | top1:98.82966613769531
111/312 Data:0.001 | Batch:0.784 | Total:0:01:26 | ETA:0:02:30 | Loss:0.031507731524405176 | top1:98.81920623779297
121/312 Data:0.012 | Batch:0.740 | Total:0:01:34 | ETA:0:02:22 | Loss:0.03164390

91/312 Data:0.001 | Batch:0.726 | Total:0:01:10 | ETA:0:02:49 | Loss:0.03306723628602513 | top1:98.7517318725586
101/312 Data:0.001 | Batch:0.772 | Total:0:01:18 | ETA:0:02:37 | Loss:0.03292993410942283 | top1:98.757568359375
111/312 Data:0.001 | Batch:0.736 | Total:0:01:25 | ETA:0:02:32 | Loss:0.03260605917354156 | top1:98.76454162597656
121/312 Data:0.001 | Batch:0.735 | Total:0:01:32 | ETA:0:02:23 | Loss:0.03248232276725375 | top1:98.7623291015625
131/312 Data:0.002 | Batch:0.759 | Total:0:01:40 | ETA:0:02:20 | Loss:0.03276903121838815 | top1:98.74565124511719
141/312 Data:0.001 | Batch:0.859 | Total:0:01:48 | ETA:0:02:13 | Loss:0.03319187854013756 | top1:98.73303985595703
151/312 Data:0.011 | Batch:0.778 | Total:0:01:56 | ETA:0:02:04 | Loss:0.03326311906096557 | top1:98.73336029052734
161/312 Data:0.001 | Batch:0.834 | Total:0:02:03 | ETA:0:01:57 | Loss:0.03331171282746407 | top1:98.73062896728516
171/312 Data:0.001 | Batch:0.730 | Total:0:02:11 | ETA:0:01:48 | Loss:0.0331849481703

...

61/312 Data:0.006 | Batch:0.980 | Total:0:00:50 | ETA:0:03:41 | Loss:0.033839952872424826 | top1:98.75059509277344
71/312 Data:0.013 | Batch:0.982 | Total:0:00:59 | ETA:0:03:37 | Loss:0.03336739820808592 | top1:98.75564575195312
81/312 Data:0.041 | Batch:0.870 | Total:0:01:08 | ETA:0:03:34 | Loss:0.03323857520741445 | top1:98.76243591308594
91/312 Data:0.002 | Batch:0.871 | Total:0:01:16 | ETA:0:03:09 | Loss:0.032883104213237105 | top1:98.76239776611328
101/312 Data:0.014 | Batch:0.884 | Total:0:01:26 | ETA:0:03:19 | Loss:0.03282510670879395 | top1:98.76477813720703
111/312 Data:0.001 | Batch:0.855 | Total:0:01:35 | ETA:0:02:56 | Loss:0.03258676404922126 | top1:98.77110290527344
121/312 Data:0.001 | Batch:1.007 | Total:0:01:44 | ETA:0:02:57 | Loss:0.032350323147579164 | top1:98.79643249511719
131/312 Data:0.013 | Batch:1.077 | Total:0:01:55 | ETA:0:03:16 | Loss:0.03250949426717658 | top1:98.78640747070312
141/312 Data:0.010 | Batch:0.892 | Total:0:02:04 | ETA:0:02:40 | Loss:0.032515333

121/312 Data:0.001 | Batch:0.742 | Total:0:01:36 | ETA:0:02:29 | Loss:0.03231456066858916 | top1:98.79643249511719
131/312 Data:0.001 | Batch:0.794 | Total:0:01:44 | ETA:0:02:20 | Loss:0.03240721898139206 | top1:98.78826141357422
141/312 Data:0.022 | Batch:0.806 | Total:0:01:52 | ETA:0:02:17 | Loss:0.03245007730888348 | top1:98.7795181274414
151/312 Data:0.003 | Batch:0.817 | Total:0:02:00 | ETA:0:02:15 | Loss:0.032449289730793196 | top1:98.78479766845703
161/312 Data:0.014 | Batch:0.785 | Total:0:02:09 | ETA:0:02:07 | Loss:0.03274309983974473 | top1:98.7758560180664
171/312 Data:0.001 | Batch:0.792 | Total:0:02:17 | ETA:0:02:02 | Loss:0.03276946041079467 | top1:98.78215026855469
181/312 Data:0.001 | Batch:0.773 | Total:0:02:26 | ETA:0:01:48 | Loss:0.032630391243989296 | top1:98.78640747070312
191/312 Data:0.001 | Batch:0.851 | Total:0:02:34 | ETA:0:01:42 | Loss:0.03251652970558998 | top1:98.78132629394531
201/312 Data:0.001 | Batch:0.723 | Total:0:02:42 | ETA:0:01:32 | Loss:0.03249890

171/312 Data:0.005 | Batch:0.792 | Total:0:02:15 | ETA:0:01:52 | Loss:0.03339711431283177 | top1:98.73814392089844
181/312 Data:0.018 | Batch:0.759 | Total:0:02:23 | ETA:0:01:40 | Loss:0.033398838703474286 | top1:98.74081420898438
191/312 Data:0.018 | Batch:0.758 | Total:0:02:30 | ETA:0:01:33 | Loss:0.03333597933551716 | top1:98.75082397460938
201/312 Data:0.003 | Batch:0.751 | Total:0:02:38 | ETA:0:01:27 | Loss:0.033064033317180413 | top1:98.76346588134766
211/312 Data:0.001 | Batch:0.724 | Total:0:02:46 | ETA:0:01:17 | Loss:0.032978497321995516 | top1:98.76914978027344
221/312 Data:0.001 | Batch:0.732 | Total:0:02:53 | ETA:0:01:09 | Loss:0.033325302039538575 | top1:98.75455474853516
231/312 Data:0.001 | Batch:0.741 | Total:0:03:01 | ETA:0:01:02 | Loss:0.03357853088124748 | top1:98.74018096923828
241/312 Data:0.001 | Batch:0.780 | Total:0:03:08 | ETA:0:00:54 | Loss:0.0335252329057925 | top1:98.74008178710938
251/312 Data:0.001 | Batch:0.721 | Total:0:03:16 | ETA:0:00:48 | Loss:0.03386

221/312 Data:0.011 | Batch:0.749 | Total:0:02:59 | ETA:0:01:11 | Loss:0.03265334147015982 | top1:98.78421020507812
231/312 Data:0.010 | Batch:0.753 | Total:0:03:07 | ETA:0:01:03 | Loss:0.0327948671342297 | top1:98.77275085449219
241/312 Data:0.011 | Batch:0.784 | Total:0:03:15 | ETA:0:00:56 | Loss:0.03270081738666998 | top1:98.7733154296875
251/312 Data:0.005 | Batch:0.874 | Total:0:03:23 | ETA:0:00:49 | Loss:0.03257058333397743 | top1:98.77384185791016
261/312 Data:0.002 | Batch:0.759 | Total:0:03:30 | ETA:0:00:39 | Loss:0.032875347443610085 | top1:98.7650146484375
271/312 Data:0.001 | Batch:0.745 | Total:0:03:38 | ETA:0:00:32 | Loss:0.03293452310509897 | top1:98.76580810546875
281/312 Data:0.013 | Batch:0.776 | Total:0:03:46 | ETA:0:00:24 | Loss:0.03293331925594828 | top1:98.77172088623047
291/312 Data:0.001 | Batch:0.758 | Total:0:03:53 | ETA:0:00:17 | Loss:0.032885578496196013 | top1:98.77306365966797
301/312 Data:0.001 | Batch:0.808 | Total:0:04:01 | ETA:0:00:09 | Loss:0.032660728

271/312 Data:0.004 | Batch:0.858 | Total:0:03:42 | ETA:0:00:33 | Loss:0.03267169169174349 | top1:98.78730010986328
281/312 Data:0.002 | Batch:0.898 | Total:0:03:51 | ETA:0:00:27 | Loss:0.03297544036100344 | top1:98.76394653320312
291/312 Data:0.008 | Batch:0.789 | Total:0:03:59 | ETA:0:00:18 | Loss:0.03294513146376999 | top1:98.76472473144531
301/312 Data:0.001 | Batch:0.807 | Total:0:04:07 | ETA:0:00:10 | Loss:0.032785391413368455 | top1:98.76786041259766
311/312 Data:0.000 | Batch:0.720 | Total:0:04:16 | ETA:0:00:01 | Loss:0.03286671397123498 | top1:98.76299285888672
81/81 Data:0.000 | Batch:0.225 | Total:0:00:31 | ETA:0:00:00 | Loss:0.017223049011777774 | top1:99.40499114990234

Epoch: [279 | 300] LR: 0.000079
1/312 Data:3.564 | Batch:4.505 | Total:0:00:04 | ETA:0:23:21 | Loss:0.03016500547528267 | top1:99.02912139892578
11/312 Data:0.002 | Batch:0.798 | Total:0:00:12 | ETA:0:06:04 | Loss:0.03715418770231984 | top1:98.69815063476562
21/312 Data:0.001 | Batch:0.911 | Total:0:00:21 | 